In [35]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [8]:
df = pd.read_csv(
    "data/Euronext_Equities_2025-08-19.csv",
    sep=";",          # Euronext uses semicolons
    skiprows=0,       # adjust if file has metadata rows
    on_bad_lines="skip"  # skip problematic rows if any
)


In [ ]:
df = df.dropna()
oslo_stocks = df.loc[df['Market'] == 'Oslo Børs']

In [10]:
oslo_stocks

,Name,ISIN,Symbol,Market,Currency,Open Price,High Price,low Price,last Price,last Trade MIC Time,Time Zone,Volume,Turnover,Closing Price,Closing Price DateTime
3,2020 BULKERS,BMG9156K1018,2020,Oslo Børs,NOK,"133,50","135,00","133,00","133,10",19/08/2025 16:25,CET,107862,"14447074,90","133,10",19/08/2025
39,ABG SUNDAL COLLIER,NO0003021909,ABG,Oslo Børs,NOK,"6,96","7,05","6,95","6,95",19/08/2025 16:25,CET,2305357,"17000246,86","6,95",19/08/2025
44,ABL GROUP,NO0010715394,ABL,Oslo Børs,NOK,"9,36","9,36","9,14","9,14",19/08/2025 16:15,CET,47463,"437236,10","9,14",19/08/2025
116,AF GRUPPEN,NO0003078107,AFG,Oslo Børs,NOK,"160,20","163,60","160,00","163,40",19/08/2025 16:25,CET,29299,"4764223,60","163,40",19/08/2025
134,AGILYX,NO0010872468,AGLX,Oslo Børs,NOK,"24,50","25,00","24,50","25,00",19/08/2025 16:25,CET,10652,"264156,40","25,00",19/08/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3987,WILH. WILHELMSEN B,NO0010576010,WWIB,Oslo Børs,NOK,"470,00","475,00","470,00","470,50",19/08/2025 16:27,CET,6414,"3026836,00","470,50",19/08/2025
4016,YARA INTERNATIONAL,NO0010208051,YAR,Oslo Børs,NOK,"376,60","378,80","374,50","375,60",19/08/2025 16:26,CET,495690,"186290912,288","375,60",19/08/2025
4024,ZALARIS,NO0010708910,ZAL,Oslo Børs,NOK,"76,00","77,00","76,00","77,00",19/08/2025 16:25,CET,882,"67622,40","77,00",19/08/2025
4025,ZAPTEC,NO0010713936,ZAP,Oslo Børs,NOK,"26,25","26,50","25,55","25,95",19/08/2025 16:27,CET,554446,"14464193,45","25,95",19/08/2025


In [37]:
oslo_stocks
oslo_tickers = list(oslo_stocks['Symbol'].values)
oslo_tickers = ['.'.join([i, 'OL']) for i in oslo_tickers]
ticker_data = []

In [38]:
oslo_tickers

['2020.OL',
 'ABG.OL',
 'ABL.OL',
 'AFG.OL',
 'AGLX.OL',
 'AIRX.OL',
 'AKAST.OL',
 'AKER.OL',
 'AKBM.OL',
 'AKRBP.OL',
 'ACC.OL',
 'AKH.OL',
 'AKSO.OL',
 'AKVA.OL',
 'AMSC.OL',
 'ARCH.OL',
 'AZT.OL',
 'AFK.OL',
 'ARR.OL',
 'ATEA.OL',
 'ASAS.OL',
 'ASA.OL',
 'AURG.OL',
 'AUSS.OL',
 'AUTO.OL',
 'AGAS.OL',
 'ACR.OL',
 'B2I.OL',
 'BAKKA.OL',
 'BGBIO.OL',
 'BEWI.OL',
 'BIEN.OL',
 'BNOR.OL',
 'BONHR.OL',
 'BOR.OL',
 'BRG.OL',
 'BOUV.OL',
 'BWE.OL',
 'BWLPG.OL',
 'BWO.OL',
 'BMA.OL',
 'CADLR.OL',
 'CAPSL.OL',
 'CAVEN.OL',
 'CRNA.OL',
 'CLOUD.OL',
 'CMBTO.OL',
 'CONTX.OL',
 'DNB.OL',
 'DNO.OL',
 'DOFG.OL',
 'EIOF.OL',
 'EMGS.OL',
 'ELK.OL',
 'ELABS.OL',
 'ELMRA.OL',
 'ELO.OL',
 'ENDUR.OL',
 'ENSU.OL',
 'ENTRA.OL',
 'ENVIP.OL',
 'EQNR.OL',
 'EQVA.OL',
 'EPR.OL',
 'FLNG.OL',
 'FRO.OL',
 'GENT.OL',
 'GJF.OL',
 'GOGL.OL',
 'GOD.OL',
 'GSF.OL',
 'GYL.OL',
 'HAFNI.OL',
 'HGSB.OL',
 'HAVI.OL',
 'HERMA.OL',
 'HEX.OL',
 'HPUR.OL',
 'HSHP.OL',
 'HBC.OL',
 'HYPRO.OL',
 'HAUTO.OL',
 'HSPG.OL',
 'IDEX.OL',

In [39]:
oslo_timeseries =  [yf.download(f"{tick}", start="2021-07-01", end="2025-06-01") for tick in oslo_tickers]
osl_time_dict = zip(oslo_tickers, oslo_timeseries)

/var/folders/3l/q_yssm517fj86sqc7nn2zrcm0000gn/T/ipykernel_58047/2357012814.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  oslo_timeseries =  [yf.download(f"{tick}", start="2021-07-01", end="2025-06-01") for tick in oslo_tickers]
[*********************100%***********************]  1 of 1 completed
/var/folders/3l/q_yssm517fj86sqc7nn2zrcm0000gn/T/ipykernel_58047/2357012814.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  oslo_timeseries =  [yf.download(f"{tick}", start="2021-07-01", end="2025-06-01") for tick in oslo_tickers]
[*********************100%***********************]  1 of 1 completed
/var/folders/3l/q_yssm517fj86sqc7nn2zrcm0000gn/T/ipykernel_58047/2357012814.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  oslo_timeseries =  [yf.download(f"{tick}", start="2021-07-01", end="2025-06-01") for tick in oslo_tickers]
[*********************100%***********************]  1 o

In [40]:
osl_time_dict = dict(zip(oslo_tickers, oslo_timeseries))

In [41]:
earliest_date = datetime.strptime("2021-07-01", "%Y-%m-%d")
latest_date = datetime.strptime("2025-05-30", "%Y-%m-%d")
filtered_timeseries = [(ticker, osl_time_dict[ticker]) for ticker in osl_time_dict.keys() if not osl_time_dict[ticker].empty]
filtered_timeseries = [(ticker, timeseries) for ticker, timeseries in filtered_timeseries if ((timeseries.index[0] == earliest_date) and (timeseries.index[-1] == latest_date))]

In [50]:
osl_time_dict['CADLR.OL']

Price,Close,High,Low,Open,Volume
Ticker,CADLR.OL,CADLR.OL,CADLR.OL,CADLR.OL,CADLR.OL
Date,,,,,
2021-07-01,36.000000,36.169998,34.360001,34.750000,657684
2021-07-02,36.590000,36.794998,36.000000,36.439999,719343
2021-07-05,35.610001,36.799999,35.505001,36.799999,192391
2021-07-06,34.799999,35.685001,34.674999,35.500000,187738
2021-07-07,34.200001,35.430000,33.759998,35.099998,216347
...,...,...,...,...,...
2025-05-23,50.000000,50.250000,47.779999,48.200001,547977
2025-05-26,48.320000,49.560001,48.139999,49.000000,717578


In [48]:
filtered_timeseries[30][1].resample('7d').asfreq().interpolate(method='linear')

Price,Close,High,Low,Open,Volume
Ticker,ROGS,ROGS,ROGS,ROGS,ROGS
Date,,,,,
2020-01-02,27.291250,27.291250,27.291250,27.291250,100.0
2020-01-09,27.120129,27.220474,27.120129,27.220474,1000.0
2020-01-16,27.692635,27.692635,27.541220,27.541220,5300.0
2020-01-23,27.463268,27.514336,27.308270,27.335149,3300.0
2020-01-30,26.668571,26.668571,26.457129,26.537764,2800.0
...,...,...,...,...,...
2025-05-01,38.888805,38.888805,38.809195,38.809195,2200.0
2025-05-08,39.712753,39.712753,39.386359,39.386359,300.0


In [45]:
tickers = [ts_tuple[0] for ts_tuple in filtered_timeseries]
averaged_ts = np.concatenate([(ts_tuple[1]['Close'].values + ts_tuple[1]['High'].values)/2 for ts_tuple in filtered_timeseries], axis = 1)

In [52]:
final_df = pd.DataFrame(averaged_ts, columns=tickers, index = filtered_timeseries[0][1].index)
final_df

,2020.OL,ABG.OL,ABL.OL,AFG.OL,AGLX.OL,AIRX.OL,AKAST.OL,AKER.OL,AKBM.OL,AKRBP.OL,...,VVL.OL,VOW.OL,WAWI.OL,WSTEP.OL,WWI.OL,WWIB.OL,YAR.OL,ZAL.OL,ZAP.OL,ZLNA.OL
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-01,107.400370,6.712875,7.573392,157.172466,30.250,8.7250,6.535,557.269078,73.449997,197.322209,...,156.917837,40.740000,24.602810,22.413370,161.273315,156.241742,356.162452,57.999274,46.180000,733.000
2021-07-02,105.275943,6.686289,7.540284,157.421021,29.500,8.5700,6.410,566.168419,72.650002,197.883260,...,156.050888,39.670000,24.306966,21.907511,160.396835,155.805922,358.059630,59.174933,46.230000,738.000
2021-07-05,104.803854,6.706228,7.540284,156.426792,29.800,9.1450,6.360,575.067723,72.000000,198.444326,...,156.050879,41.699999,23.987139,21.985337,157.548248,156.023864,362.744460,58.978990,44.850000,750.000
2021-07-06,106.220113,6.766046,7.614775,156.178232,30.250,9.2450,6.290,580.364960,73.750000,198.023497,...,155.617404,41.430000,23.875200,22.179897,159.520355,157.331314,361.079592,59.566818,43.940001,740.000
2021-07-07,107.636415,6.752754,7.490623,156.592484,28.950,8.9950,6.295,572.525118,73.550003,192.377712,...,155.183929,42.959999,23.611342,21.596214,161.930679,157.767136,363.634925,59.468845,42.949999,735.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-23,118.944981,6.675000,10.140000,149.300003,29.100,1.1825,11.890,597.000000,59.549999,228.401463,...,364.000000,2.220000,81.575001,25.500000,419.500000,402.250000,374.124131,82.299999,20.725000,13.426
2025-05-26,119.693372,6.640000,10.125000,146.699997,28.275,1.2500,12.140,601.000000,59.799999,229.814360,...,366.000000,2.820000,81.900002,25.600000,421.000000,402.250000,373.334741,81.399998,20.825000,13.649
2025-05-27,118.845196,6.650000,10.200000,148.000000,27.975,1.3000,12.080,601.000000,59.549999,231.373416,...,364.000000,2.675000,81.099998,25.400001,416.250000,398.000000,374.272125,80.599998,20.325000,13.335


In [ ]:
# Filter by volatility
# Filter by market cap